# Assignment 3 (Forest Cover)

- Name: **Arnab Sen**
- Roll: **510519006**
- Date: **Sept 5, 2022**

## Task 5

In [ ]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [ ]:
drive.mount('/content/drive')
BASE_PATH = '/content/drive/MyDrive/Colab_Notebooks/ML_DRIVE/Assign_3/dataset'

In [ ]:
dataset = pd.read_csv(f"{BASE_PATH}/covtype.csv")
print("Dataset shape:", dataset.shape)
print("Dataset columns:", dataset.columns)

In [ ]:
# Let's check for missing values once
dataset.isnull().sum()

In [ ]:
dataset.info()

In [ ]:
scaled_cols = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
'Horizontal_Distance_To_Fire_Points']
for col in scaled_cols:
  scaler = StandardScaler()
  dataset[[col]] = pd.DataFrame(
    data=scaler.fit_transform(dataset[[col]]),
    index=dataset.index,
    columns=[col]
  )

In [ ]:
def plot_count(y):
  before_dist = Counter(y)
  print("Before undersampling: ", before_dist)
  plt.xlabel("Count")
  plt.ylabel("Cover Type")
  plt.title("Count of each cover type")
  plt.bar(before_dist.keys(), before_dist.values())

In [ ]:
X = dataset.drop('Cover_Type', axis = 1)
y = dataset['Cover_Type']

plot_count(y)

In [ ]:
# define undersampling strategy
undersample = RandomUnderSampler(sampling_strategy='not minority')
X, y = undersample.fit_resample(X, y)
plot_count(y)

In [ ]:
X_train, _X, y_train, _y = train_test_split(X, y, train_size=0.8)

X_test, X_val, y_test, y_val = train_test_split(_X, _y, train_size = 0.5)

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
cm = confusion_matrix(y_test, pred)
f1 = f1_score(y_test, pred, average='macro')
accuracy = clf.score(X_test, y_test)
print(f"Accuracy = {accuracy}\n")
print(f"F1 Score = {f1}\n")

## Task 6

In [ ]:
sns.heatmap(cm)
plt.title('HeatMap')
plt.ylabel('Actual Value')
plt.xlabel('Predicted Value')
plt.show()

In [ ]:
sub_X_train = X.iloc[:, 0:2]
sub_y_train = y

sub_train = sub_X_train.join(sub_y_train)
sub_train = sub_train[sub_train['Cover_Type'].isin([1,2,3])]

sub_X = sub_train.drop('Cover_Type', axis = 1)
sub_y = sub_train['Cover_Type']


In [ ]:
plot_count(sub_y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sub_X, sub_y, train_size=0.8)

In [ ]:
regr = LogisticRegression(multi_class='multinomial')
model = regr.fit(X_train, y_train)
y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)
print(f"Accuracy (Test) = {model.score(X_test, y_test)}")
print(f"Accuracy (Train) = {model.score(X_train, y_train)}")
print(f"F1 Score (Test)= {f1_score(y_test, y_pred_test, average='macro')}")
print(f"F1 Score (Train)= {f1_score(y_train, y_pred_train, average='macro')}")

In [ ]:
df_train = X_train
df_train['CoverType'] = y_train
df_train

In [ ]:
df_test = X_test
df_test['CoverType'] = y_test
df_test

In [ ]:
df_pred_test = X_test
df_pred_test['CoverType'] = y_pred_test
df_pred_test

In [ ]:
df_pred_train = X_train
df_pred_train['CoverType'] = y_pred_train
df_pred_train

In [ ]:
df_trains = [df_train[df_train['CoverType'] == i] for i in [1, 2, 3]]
df_tests = [df_test[df_test['CoverType'] == i] for i in [1, 2, 3]]
df_pred_tests = [df_pred_test[df_pred_test['CoverType'] == i] for i in [1, 2, 3]]
df_pred_trains = [df_pred_train[df_pred_train['CoverType'] == i] for i in [1, 2, 3]]

## Task 7

In [ ]:
def plot_scatter(title, dfs):
  plt.xlabel("Aspect")
  plt.ylabel("Elevation")
  plt.title(title)
  for _df in dfs:
    plt.scatter(_df['Aspect'], _df['Elevation'])

In [ ]:
plot_scatter("Train", df_trains)

In [ ]:
plot_scatter("Train (Predicted)", df_pred_trains)

In [ ]:
plot_scatter("Test", df_tests)

In [ ]:
plot_scatter("Test (Predicted)", df_pred_tests)